# Notebook to match VASCA sources to SIMBAD database sources positionally
Set your option in the cell bellow

In [ ]:
from astropy import units as uu
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_cat_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"

In [ ]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from vasca.region import Region
import vasca.visualization as vvis
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, get_col_cycler,add_ogrp, color_palette, nb_fig

# Get region
rc = Region()
rc.load_from_fits(region_cat_fname)

### Prepare association data for plotting

In [ ]:
#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp","match_distance"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_simbad")
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
sel_simbad = rc.tt_sources["sel"].data
print(f"Number of SIMBAD matches is: {sel_simbad.sum()}  ({100* sel_simbad.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiadr3",["PQSO", "PGal", "PSS","match_distance","Gmag_abs","BP-RP", "ogrp"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_gaiadr3")
rc.tt_sources.rename_column("ogrp","ogrp_gaiadr3")
sel_gaiadr3 = rc.tt_sources["sel"].data
print(f"Number of GAIA matches is: {sel_gaiadr3.sum()}  ({100*sel_gaiadr3.sum() / len(rc.tt_sources)} )%")

sel_match = np.array(sel_simbad+sel_gaiadr3, dtype=bool)
print(f"Total number of matches is: {sel_match.sum()}  ({100*sel_match.sum() / len(rc.tt_sources)} )%")

#Copy lomb scargle result into tt_sources
rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_model_rchiq"],"rg_src_id")

display(rc.tt_sources)
#display(rc.tt_simbad)


### Create matching table

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
axs = np.array(axs).flatten()
rc.tt_sources["sel"]=sel_simbad
_ = vvis.plot_table_hist(rc.tt_sources[sel_simbad], "match_distance_simbad", ax=axs[0], logx=False, obs_filter_id=None, density= False)
rc.tt_sources["sel"]=sel_gaiadr3
_ = vvis.plot_table_hist(rc.tt_sources[sel_gaiadr3], "match_distance_gaiadr3", ax=axs[1], logx=False, obs_filter_id=None, density= False)
rc.tt_sources["sel"]=sel_match
_ = vvis.plot_table_hist(rc.tt_sources, 'pos_err', ax=axs[2], logx=False, obs_filter_id=None, density= False)

### Plot found object types

In [ ]:
#Prepare data
sel_mt = rc.tt_sources["sel"]
otypes_all, otype_cts_all= np.unique(rc.tt_sources[sel_mt]["otype"],return_counts=True)
dd_otype2id = dict(zip(list(otypes_all),range(len(otypes_all))))

#Do bar chart of all otypes color coded by ogrp
fig_otypes = plt.figure("Source types",figsize=(18, 7), clear=True)
fig_otypes.clf()
tt_mt_grp =  rc.tt_sources[sel_mt].group_by("ogrp_simbad")
for group, tt_grp in zip(tt_mt_grp.groups.keys, tt_mt_grp.groups):
    otypes, otype_cts= np.unique(tt_grp["otype"],return_counts=True)
    otypes_id = []
    for otype in otypes:
        otypes_id.append(dd_otype2id[otype])
    plt.bar( otypes_id, otype_cts, align='center', label=group[0], color = dd_ogrp2col[group[0]])
plt.yscale('log')
#plt.grid()
plt.xlabel("Object type")
plt.ylabel("Nr of sources")
plt.show()
plt.legend()
_ = plt.xticks(range(len(otypes_all)), otypes_all, size='small')

#Printout otypes whch have no ogrp assigned yet
sel_ogrp_none = rc.tt_simbad["ogrp"]=="none"
if sel_ogrp_none.sum()>0:
    print("Unclassified new otypes:")
    display(rc.tt_simbad["main_id","otype","rg_src_id"][sel_ogrp_none])

In [ ]:
# Pie chart showing fractions of grouped classification

# Grouped classifications
sel_mt = rc.tt_sources["sel"]
plabel, pvals = np.unique(rc.tt_sources[sel_mt]["ogrp_simbad"],return_counts=True)
percent = [i/pvals.sum()*100 for i in pvals]

# Figure (notebook-optimized)
fig, ax = nb_fig(num="class pie", gr_size=8, layout="constrained")

# Seaborn colors
alpha = 0.95
c_bright_rgb = color_palette(name="bright", n=len(plabel), show_in_notebook=False)
c_bright_rgba = [(*c[:3], alpha) for c in c_bright_rgb]

# Chart
patches, text = ax.pie(pvals,
					   labels=plabel,
					   colors=c_bright_rgba,
					   startangle=270,
					   radius=1.2,
					   textprops={'fontsize': 7}
					  )

# External legend, sorted by percentage
xlabels = [f'{i} - {j:d} ({k:1.1f}%)' for i,j,k in zip(plabel, pvals, percent)]
sort_legend = True
if sort_legend:
	patches, labels, _ =  zip(
		*sorted(
			zip(patches, xlabels, pvals),
			key=lambda x: x[2],
			reverse=True
		)
	)
_=ax.legend(
	patches,
	labels,
	loc='best',
	bbox_to_anchor=(-0.1, 1.),
	fontsize=10,
	title=f"Number of cross-matched\nVASCA sources (total: {pvals.sum()})",
	title_fontsize=12,
	alignment='left',
)

### Scatter plots of source variables by group

In [ ]:
plot_kwargs = {"markersize": 3.0,"alpha":0.6}
fig, axs = plt.subplots(2, 4, figsize=(19, 8))
#fig.clf()
axs = axs.flatten()
plt_ogrops = [["AGN","UNK","GAL",'Star*'],["AGN",'WD*', 'B*']]

plt_flts = [1,2]
ctr = 0
for flt in plt_flts:
    for ogrps in plt_ogrops:
        axs[ctr].set_prop_cycle(get_col_cycler(ogrps))
        vvis.plot_table_scatter(rc.tt_sources,"flux_nxv", "flux", ax=axs[ctr], xscale="log",yscale="log", obs_filter_id=flt, grp_var="ogrp_simbad",grp_vals = ogrps,**plot_kwargs)

        axs[ctr+4].set_prop_cycle(get_col_cycler(ogrps))
        vvis.plot_table_scatter(rc.tt_sources,"flux_nxv", "hr", ax=axs[ctr+4], xscale="log",yscale="log", obs_filter_id=flt, grp_var="ogrp_simbad",grp_vals = ogrps,**plot_kwargs)
        ctr +=1

_ = axs[0].legend()
_ = axs[1].legend()
_ = axs[2].legend()
_ = axs[3].legend()
#print(dd_ogrp2otypes.keys())r

### Compare object classification from SIMBAD to the one from GAIA-DR3

In [ ]:
#Reset source selection
rc.tt_sources["sel"]=True

#Define selections for different SIMBAD object groups
sel_AGN = rc.tt_sources["ogrp_simbad"]=="AGN"
sel_star = np.array((rc.tt_sources["ogrp_simbad"]=="Star*")+(rc.tt_sources["ogrp_simbad"]=="S*"), dtype=bool)
sel_gal = rc.tt_sources["ogrp_simbad"]=="GAL"
sels_ogrp = [sel_AGN,sel_star,sel_gal]

#GAIA variables to plot
gaia_vars = ["PQSO","PSS","PGal"]

#Define figure parameters and loop to do plots
fig, axs = plt.subplots(1, 3, figsize=(18, 6),num="Cummulative probabilities of object groups")
hist_bins = np.linspace(0.0,1.0,1001)
hist_dens = True
axs = np.array(axs).flatten()
panel = 0
cumulative = -1
for var in gaia_vars:
    for sel in sels_ogrp:
        _ = vvis.plot_table_hist(rc.tt_sources[sel], var, ax=axs[panel], density= hist_dens, bins=hist_bins,cumulative= cumulative )
    panel+=1
    
#Print out figure labels
print("SIMBAD assoc: orange:AGN; red:Star; brown:Galaxy")

### Hertzsprung russel diagram

In [ ]:
tt_HR = rc.tt_sources[rc.tt_sources["Gmag_abs"]>-10]
fig, axs = plt.subplots(1, 1, figsize=(5, 8),num="HR", clear=True)
axs = np.array(axs).flatten()
axs[0].invert_yaxis()
axs[0].plot(tt_HR["BP-RP"],tt_HR["Gmag_abs"],"o")
#axs[0].invert_yaxis()
axs[0].plot(tt_HR["BP-RP"][tt_HR["ogrp_gaiadr3"]=="WD*"],tt_HR["Gmag_abs"][tt_HR["ogrp_gaiadr3"]=="WD*"],"o")
#axs[0].invert_yaxis()
axs[0].set_xlabel("Gaia BP-RP color")
axs[0].set_ylabel("Gaia G absolute magnitude")
plt.show()

### Print info for selected sources

In [ ]:
# Make selection
sel_otype = rc.tt_sources["otype"] ==  "RR*"# "No*"#"HS?" #"BS*" #"PM*" # # "SN*" #"EB*"
sel_otypes = (rc.tt_sources["otype"] ==  "WD?")  + (rc.tt_sources["otype"] =="WD*") + (rc.tt_sources["ogrp_gaiadr3"] =="WD*")
sel_flux_nxv = rc.tt_sources["flux_nxv"][:,0]>2
sel_flux_hr = rc.tt_sources["hr"]>2
sel_src_id = rc.tt_sources["rg_src_id"]==15179
sel_period = (rc.tt_sources["ls_peak_pval"]>-0.5) * (rc.tt_sources["ls_peak_pval"]<0.000000573303) * (rc.tt_sources["ls_model_rchiq"]<1.1)# # 3 sigma 0.002699796063, 4 sigma 0.000063342484, 5 sigma 0.000000573303
sel_srcs =  sel_period #sel_otypes  #sel_src_id  #sel_flux_nxv #sel_otype # sel_flux_nxv_fuv # sel_flux_hr#

#Display match and simbad table for selected sources
display(rc.tt_sources[sel_srcs])

#Print oout list of source IDs to be used for further use, e.g. in inspect_sources.ipynb
print("srcs_ids = [", end="")
for src_id in rc.tt_sources[sel_srcs]["rg_src_id"].data:
    print(src_id,",", end="")
print("]")
print("Total Nr. of sources:",sel_srcs.sum())

In [ ]:
tt_nodes = Table.read("./resources/SIMBAD_otypes/otypes_nodes3.csv")
tt_nodes.rename_column("Id", "otype")
#display(tt_nodes)
add_ogrp(tt_nodes)
tt_nodes["otype","Candidate","Description","ogrp","Label","Category","Subcategory"].pprint_all()

In [ ]:
from astroquery.simbad import Simbad

testSimbad = Simbad()

#testSimbad.list_votable_fields()
all_vo_fields = ["biblio","cel","cl.g","coo(opt)","coo_bibcode","coo_err_angle","coo_err_maja","coo_err_mina","coo_qual","coo_wavelength","coordinates","dec(opt)","dec_prec","diameter","dim","dim_angle","dim_bibcode","dim_incl","dim_majaxis","dim_minaxis","dim_qual","dim_wavelength","dimensions","distance","distance_result","einstein","fe_h","flux(filtername)","flux_bibcode(filtername)","flux_error(filtername)","flux_name(filtername)","flux_qual(filtername)","flux_system(filtername)","flux_unit(filtername)","fluxdata(filtername)","gcrv","gen","gj","hbet","hbet1","hgam","id(opt)","ids","iras","irc","iso","iue","jp11","link_bibcode","main_id","measurements","membership","mesplx","mespm","mk","morphtype","mt","mt_bibcode","mt_qual","otype","otype(opt)","otypes","parallax","plx","plx_bibcode","plx_error","plx_prec","plx_qual","pm","pm_bibcode","pm_err_angle","pm_err_maja","pm_err_mina","pm_qual","pmdec","pmdec_prec","pmra","pmra_prec","pos","posa","propermotions","ra(opt)","ra_prec","rot","rv_value","rvz_bibcode","rvz_error","rvz_qual","rvz_radvel","rvz_type","rvz_wavelength","sao","sp","sp_bibcode","sp_nature","sp_qual","sptype","td1","typed_id","ubv","uvby","uvby1","v*","velocity","xmm","z_value"]

#for vo_field in all_vo_fields:
#    print("*",vo_field)
#    testSimbad.get_field_description(vo_field)